# Inférence Statistique

### Anova

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import levene, shapiro, f_oneway
from scipy.stats import chi2_contingency
import scipy.stats as stats
from scipy.stats import pearsonr

In [2]:
data = pd.read_csv("data/dataset_clean.csv")
data["_<1H OCEAN"] = data["_<1H OCEAN"].apply(str)
data.head()

,longitude,latitude,Age moyen logement,Total pieces,Total chambres,Nb personne logement,Nb famille logement,Salaire median logement,Valeur moyenne logement,_<1H OCEAN,_INLAND,_ISLAND,_NEAR BAY,_NEAR OCEAN
0,-119.84,36.77,6.0,1853.0,473.0,1397.0,417.0,1.4817,72000.0,0,1,0,0,0
1,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,1,0,0,0,0
2,-120.19,36.60,25.0,875.0,214.0,931.0,214.0,1.5536,58300.0,0,1,0,0,0
3,-118.32,34.10,31.0,622.0,229.0,597.0,227.0,1.5284,200000.0,1,0,0,0,0
4,-121.23,37.79,21.0,1922.0,373.0,1130.0,372.0,4.0815,117900.0,0,1,0,0,0


#### On commence par le Test Levene. 

On peut constater que la condition n'est pas respectée, nous pouvons donc pas appliquer le test Anova

In [3]:
Alpha = 0.05
stat, p = levene(
                data["Valeur moyenne logement"][data["_<1H OCEAN"] == "0"],
                data["Valeur moyenne logement"][data["_<1H OCEAN"] == "1"]
                )
if not p > Alpha:
    print("Levene : Variance non similaire !")
else:
    print("Levene : Variance similaire")
    print(f"P-value = {p}")

Levene : Variance non similaire !


### Test Pearson

On souhaite réaliser un Test Pearson pour voir la corréliation entre les colonnes qui nous paraissent similaire : <br>
=> Total pieces, Total chambres <br>
=> Nb personne logement, Nb famille logement

In [4]:
def pearson(X,Y):
    corr, p_value = pearsonr(X,Y)
    return print(f"Correlation : {corr}, pvalue : {p}")

In [5]:
pearson(data["Total pieces"], data["Total chambres"])

Correlation : 0.9289959669568648, pvalue : 0.0007158357204979268


In [6]:
pearson(data["Nb personne logement"], data["Nb famille logement"])

Correlation : 0.9076469213392173, pvalue : 0.0007158357204979268


Comme on peut le constater, la corrélation est forte. Nous pourrons donc à l'avenir conserver 1 colonne de chaque paire.

#### On veut maintenanr voir si la target et corréler avec la variable Salaire median logement

In [7]:
pearson(data["Valeur moyenne logement"], data["Salaire median logement"])

Correlation : 0.6896592006096155, pvalue : 0.0007158357204979268


#### On veut maintenant voir si les variables catégorielles sont corréler entre-elles

In [8]:
def KHI2(X, Y):
    alpha = 0.05
    contingency_table = pd.crosstab(X, Y)
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    print(f"Statistique de KHI2: {chi2}")
    print(f"p-value: {p}")
    if p > alpha:
        return print(f"Les données sont indépendantes")
    return print(f"Les données ne sont pas indépendantes")

In [9]:
KHI2(data["_<1H OCEAN"], data["_ISLAND"])

Statistique de KHI2: 1.6380564107012678
p-value: 0.20059234501336845
Les données sont indépendantes


In [10]:
KHI2(data["_NEAR BAY"], data["_NEAR OCEAN"])

Statistique de KHI2: 303.4678584616521
p-value: 5.784369480607379e-68
Les données ne sont pas indépendantes
